In [0]:
import pandas as pd
import numpy as np
import time
import datetime
import json

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/pronóstico de bolsas/data.csv', sep=',',header=0, encoding = "ISO-8859-1")
df.head()

,Fecha,Cliente,No Bolsas
0,1/05/2019,CLINICA EL GOLF,1
1,1/05/2019,HOSP. DOS DE MAYO,1
2,1/05/2019,CLINICA MONTESUR,1
3,1/05/2019,VESALIO,1
4,1/05/2019,HOSP. BARTON,1


In [0]:
labels = [c for c in df.columns.array]
data = df.values
keys = {}

In [0]:
def string_to_date(date):
  return datetime.datetime.strptime(date, "%d/%m/%Y")

def date_to_timestamp(date):
  return int(time.mktime(date.timetuple()))

def timestamp_to_string(date):
  return datetime.datetime.fromtimestamp(date).strftime("%d/%m/%Y")

def next_day(date):
  return date + datetime.timedelta(days=1)

def days_between(d1, d2):
    return abs((d2 - d1).days)

data[:,0] = [string_to_date(d) for d in data[:,0]]
data

array([[datetime.datetime(2019, 5, 1, 0, 0), 'CLINICA EL GOLF', 1],
       [datetime.datetime(2019, 5, 1, 0, 0), 'HOSP. DOS DE MAYO', 1],
       [datetime.datetime(2019, 5, 1, 0, 0), 'CLINICA MONTESUR', 1],
       ...,
       [datetime.datetime(2019, 12, 14, 0, 0), 'INCOR', 1],
       [datetime.datetime(2019, 12, 14, 0, 0), 'CLINICA TEZZA', 1],
       [datetime.datetime(2019, 12, 14, 0, 0), 'CLINICA TEZZA', 1]],
      dtype=object)

In [0]:
hospitals = []
keys[labels[1]] = []
for hospital in data[:,1]:
  if hospital not in hospitals:
    hospitals.append(hospital)
    keys[labels[1]].append(hospital)
data[:,1] = [hospitals.index(hospital) for hospital in data[:,1]]

In [0]:
data

array([[datetime.datetime(2019, 5, 1, 0, 0), 0, 1],
       [datetime.datetime(2019, 5, 1, 0, 0), 1, 1],
       [datetime.datetime(2019, 5, 1, 0, 0), 2, 1],
       ...,
       [datetime.datetime(2019, 12, 14, 0, 0), 9, 1],
       [datetime.datetime(2019, 12, 14, 0, 0), 16, 1],
       [datetime.datetime(2019, 12, 14, 0, 0), 16, 1]], dtype=object)

In [0]:
labels_proc = ["timestamp", "nbolsas"]

days = set(data[:, 0])

prev = []
for d in data:
    prev.append([d[0], d[2]])

n_days = days_between(max(days), min(days))
ev = next_day(min(days))

init = 0
while init < n_days:
  if ev not in days:
    prev.append([ev, 0])
  ev = next_day(ev)
  init += 1

prev = np.array(prev)
days = set(prev[:,0])

post = []

for d in days:
  ev = prev[prev[:,0]==d]
  row = [date_to_timestamp(d), ev[:,1].sum()]
  post.append(row)

post = np.array(post)
post = post[post[:,0].argsort()]
post = np.array([[timestamp_to_string(p[0]), p[1]] for p in post])
  
data_proc = [post]

In [0]:
data_proc = np.array(data_proc[0])

In [0]:
data_proc[:10]

array([['01/05/2019', '34'],
       ['02/05/2019', '38'],
       ['03/05/2019', '40'],
       ['04/05/2019', '73'],
       ['05/05/2019', '0'],
       ['06/05/2019', '35'],
       ['07/05/2019', '35'],
       ['08/05/2019', '36'],
       ['09/05/2019', '33'],
       ['10/05/2019', '37']], dtype='<U10')

In [0]:
path = "/content/drive/My Drive/Colab Notebooks/pronóstico de bolsas/data-proc-total.csv"
np.savetxt(path, data_proc, delimiter=",", fmt='%s')